In [2]:
import pandas as pd
from datetime import date
import datetime
import time

In [ ]:
% cd ..

# Load data

In [3]:
sensor_df = pd.read_csv('../Data/Pedestrian_Counting_System_-_Sensor_Locations.csv')
sensor_df.head(2)

,sensor_id,sensor_description,sensor_name,installation_date,status,note,direction_1,direction_2,latitude,longitude,location
0,16,Australia on Collins,Col270_T,2009/03/30,R,Device moved to location ID 53 (22/09/2015),NaN,NaN,-37.815734,144.96521,"(-37.81573422, 144.96521044)"
1,50,Faraday St-Lygon St (West),Lyg309_T,2017/11/30,A,NaN,South,North,-37.798082,144.96721,"(-37.79808191, 144.96721014)"


In [5]:
count_df = pd.read_csv('../Data/Pedestrian_Counting_System_-_Monthly__counts_per_hour_.csv')
count_df['Date_Time'] = pd.to_datetime(count_df['Date_Time'], format='%B %d, %Y %X %p')
count_df['Date'] = count_df['Date_Time'].dt.date
count_df['Month'] = pd.to_datetime(count_df['Month'], format='%B').dt.month
count_df.head(2)

,ID,Date_Time,Year,Month,Mdate,Day,Time,Sensor_ID,Sensor_Name,Hourly_Counts,Date
0,2887628,2019-11-01 05:00:00,2019,11,1,Friday,17,34,Flinders St-Spark La,300,2019-11-01
1,2887629,2019-11-01 05:00:00,2019,11,1,Friday,17,39,Alfred Place,604,2019-11-01


In [6]:
count_df.shape

(4463670, 11)

# Top 10 (most pedestrians) locations by day

In [4]:
start_time = time.time()

In [5]:
# Sum number of Pedestrian
sumPerDay = count_df.groupby(['Year', 'Month', 'Mdate', 'Sensor_ID'])\
    ['Hourly_Counts'].sum('Hourly_Counts').reset_index()
# Sort value by total Hourly Counts per day
sumPerDaySorted = sumPerDay.sort_values(['Year', 'Month', 'Mdate', 'Hourly_Counts'], 
                                        ascending = [True, True, True, False])
# Get top 10 pedestrians location by day
top10PerDay = sumPerDaySorted.groupby(['Year', 'Month', 'Mdate']).head(10)
top10PerDay.head(3)

,Year,Month,Mdate,Sensor_ID,Hourly_Counts
2,2009,5,1,4,45185
0,2009,5,1,1,36869
4,2009,5,1,6,29015


## Record

In [6]:
tmp = top10PerDay.join(sensor_df[['sensor_description', 'sensor_name', 'sensor_id']]\
    .set_index('sensor_id'), 
                       on='Sensor_ID')
fullTop10PerDay = tmp.sort_values(['Year', 'Month', 'Mdate', 'Hourly_Counts'],
                                  ascending = [True, True, True, False])
fullTop10PerDay.to_csv('Pandas_Top10PerDay.csv', index = False)

fullTop10PerDay.head(3)

,Year,Month,Mdate,Sensor_ID,Hourly_Counts,sensor_description,sensor_name
2,2009,5,1,4,45185,Town Hall (West),Swa123_T
0,2009,5,1,1,36869,Bourke Street Mall (North),Bou292_T
4,2009,5,1,6,29015,Flinders Street Station Underpass,FliS_T


In [7]:
print('Running Top 10 Per Day')
print("--- %s seconds ---" % (time.time() - start_time))

Running Top 10 Per Day
--- 1.3969950675964355 seconds ---


# Top 10 (most pedestrians) locations by month

In [8]:
start_time = time.time()

In [9]:
# Sum number of Pedestrian
sumPerMonth = count_df.groupby(['Year', 'Month', 'Sensor_ID'])['Hourly_Counts'].sum('Hourly_Counts').reset_index()
# Sort value by total Hourly Counts per day
sumPerMonthSorted = sumPerMonth.sort_values(['Year', 'Month', 'Hourly_Counts'], ascending = [True, True, False])
# Get top 10 pedestrians location by day
top10PerMonth = sumPerMonthSorted.groupby(['Year', 'Month']).head(10)
top10PerMonth.head(5)

,Year,Month,Sensor_ID,Hourly_Counts
3,2009,5,4,1095125
0,2009,5,1,842470
5,2009,5,6,729966
4,2009,5,5,711896
1,2009,5,2,628472


## Record

In [10]:
tmp = top10PerMonth.join(sensor_df[['sensor_description', 'sensor_name', 'sensor_id']]\
    .set_index('sensor_id'), on='Sensor_ID')
fullTop10PerMonth = tmp.sort_values(['Year', 'Month', 'Hourly_Counts'],
                                  ascending = [True, True, False])
fullTop10PerMonth.to_csv('Pandas_Top10PerMonth.csv', index = False)
fullTop10PerMonth.head(3)

,Year,Month,Sensor_ID,Hourly_Counts,sensor_description,sensor_name
3,2009,5,4,1095125,Town Hall (West),Swa123_T
0,2009,5,1,842470,Bourke Street Mall (North),Bou292_T
5,2009,5,6,729966,Flinders Street Station Underpass,FliS_T


In [11]:
print('Running Top 10 Per Month')
print("--- %s seconds ---" % (time.time() - start_time))

Running Top 10 Per Month
--- 0.9079995155334473 seconds ---


# Which location has shown most decline due to lockdowns in last 2 years

In [12]:
start_time = time.time()

In [13]:
# Sum number Pedestrians in 3 year 2020, 2021, 2022
countLast3year_df = count_df[count_df['Year'].isin([2020,2021,2022])]
tmp = countLast3year_df.groupby(['Year', 'Sensor_ID'])[['Hourly_Counts']].sum().reset_index('Year')
# Calculate Number of Pedestrians change from 2020 to 2022
year2020 = tmp[tmp['Year'] == 2020]['Hourly_Counts']
year2021 = tmp[tmp['Year'] == 2021]['Hourly_Counts']
year2022 = tmp[tmp['Year'] == 2022]['Hourly_Counts']

Sensor in 2020 not in 2022

In [14]:
sensor2022 = set(year2022.index)
sensor2021 = set(year2021.index)
sensor2020 = set(year2020.index)
list(sensor2020-sensor2022)

[34, 22, 15]

Sensor in 2020 not in 2021

In [15]:
list(sensor2020-sensor2021)

[]

Sensor in 2021 not in 2022

In [16]:
list(sensor2021-sensor2022)

[34, 22, 15]

In [17]:
# Calculate Number of Pedestrians change from 2020 to 2022
year2020to2022 = year2020-year2022
year2020to2022_index = set(year2020to2022[year2020to2022.notnull()].index)
year2020to2022 = year2020to2022[year2020to2022.notnull()]

In [18]:
# Calculate Number of Pedestrians change from 2020 to 2021 and not in from 2020 to 2022
year2020to2021 = year2020-year2021
year2020to2021_index = set(year2020to2021[year2020to2021.notnull()].index)
justIn20_21_index = list(set(year2020to2021_index) - set(year2020to2022_index))
justIn20_21 = year2020to2021[justIn20_21_index]
justIn20_21

Sensor_ID
34     148705.0
22    1507594.0
15    -712874.0
Name: Hourly_Counts, dtype: float64

In [19]:
# Calculate Number of Pedestrians change from 2021 to 2022 and not in from 2020 to 2022
year2021to2022 = year2021-year2022
# Get list index of value just in 2021
year2021to2022_index = set(year2021to2022[year2021to2022.notnull()].index)
justIn21_22_index = list(set(year2021to2022_index) - set(year2020to2022_index))
justIn21_22 = year2021to2022[justIn21_22_index]
justIn21_22

Sensor_ID
72    -136961.0
75       2197.0
76     102432.0
77     -28753.0
78     -67258.0
79   -1673189.0
Name: Hourly_Counts, dtype: float64

In [20]:
# Concat result
ChangeFrom2020To2022 = pd.concat([year2020to2022, justIn20_21, justIn21_22])
idx = ChangeFrom2020To2022.idxmax()
# sensorName = countLast3year_df[countLast3year_df['Sensor_ID'] == idx]['Sensor_Name'].unique()[0]
# mostDeclineDict = {
#     'Year':[2020, 2021, 2022],
#     'Sensor_ID': idx,
#     'Sum_Hourly_Count': [year2020[idx], year2021[idx], year2022[idx]]}
mostDeclineDict = {
    'Sensor_ID': [idx],
    'Sum_Hourly_Count': [year2020[idx]- year2022[idx]]}
mostDeclide_df = pd.DataFrame.from_dict(mostDeclineDict)
mostDeclide_df

,Sensor_ID,Sum_Hourly_Count
0,55,3815915


## Record

In [21]:
# Record result
mostDeclide = mostDeclide_df.join(sensor_df[['sensor_description', 'sensor_name', 'sensor_id']]\
    .set_index('sensor_id'), on='Sensor_ID')

mostDeclide.to_csv('Pandas_MostDeclineLast2Year.csv', index = False)
mostDeclide

,Sensor_ID,Sum_Hourly_Count,sensor_description,sensor_name
0,55,3815915,Elizabeth St-La Trobe St (East),Eli380_T


In [22]:
print('Running Most Decline Last 2 Year')
print("--- %s seconds ---" % (time.time() - start_time))

Running Most Decline Last 2 Year
--- 2.160001754760742 seconds ---


# Which location has most growth in last year

In [23]:
start_time = time.time()

In [24]:
res = year2021 - year2022
# Get index of location most growth last year
idx = res.idxmin()
mostGrowth = {'Sensor_ID': [idx],
              'Hourly_Count': [year2021[idx] - year2021[idx]]}
mostGrowth_df = pd.DataFrame.from_dict(mostGrowth)
mostGrowth_df

,Sensor_ID,Hourly_Count
0,3,814767


## Record

In [25]:
# Record result
mostGrowth = mostGrowth_df.join(sensor_df[['sensor_description', 'sensor_name', 'sensor_id']]\
    .set_index('sensor_id'), on='Sensor_ID')

mostGrowth.to_csv('Pandas_MostGrowthLastYear.csv', index = False)
mostGrowth

,Sensor_ID,Hourly_Count,sensor_description,sensor_name
0,3,814767,Melbourne Central,Swa295_T


In [26]:
print('Running Most Growth last year')
print("--- %s seconds ---" % (time.time() - start_time))

Running Most Growth last year
--- 0.309002161026001 seconds ---
